In [1]:
import sys
import os
sys.path.append('C:\\Users\oscar\OneDrive\Dokumenter\Høst 2023\TET4565 Spesialiseringsemne\Hydro_optimization') #OSCAR path
sys.path.append('C:\\Users\\benny\\Documents\\Preliminary-project')  #BENJAMIN path
sys.path.append('C:\\Users\\Epsilon Delta\\OneDrive - NTNU\\Energi og Miljø NTNU\\Høst2023\\TET4565 Fordypningsemne\\Hydro_optimization') #ESPEN path

import pyomo.environ as pyo
import numpy as np
from pyomo.environ import ConcreteModel,Set,RangeSet,Param,Suffix,Reals,NonNegativeReals,NonPositiveReals,Binary,Objective,minimize,maximize,value
from pyomo.core import Constraint,Var,Block,ConstraintList
from pyomo.opt import SolverFactory, SolverStatus, TerminationCondition
from pyomo.util.infeasible import log_infeasible_constraints
import matplotlib.pyplot as plt
from calculations.datahandling import*
from calculations.data_processor import* 

## Data handling

In [2]:
# ---------------------------- Read in parameters and hydro topology -------------------------------------- # WE WAIT WITH THIS UNTIL LATER

def InputData(data_file):
    inputdata = pd.read_excel(data_file)
    inputdata = inputdata.set_index('Parameter', drop=True)
    inputdata = inputdata.transpose()
    data = {}
    data['prod'] = inputdata[['Ci', 'yi', 'P_min', 'P_max']]
    data['prod'].drop('Solar', inplace=True)
    return data

parameters=InputData('data/Parameters.xlsx')


def InputData(data_file):
    inputdata = pd.read_excel(data_file, sheet_name='Hydro')
    inputdata = inputdata.set_index('Ormsetfoss', drop=True)
    inputdata = inputdata.transpose()
    data = {}
    data['ormset_prod'] = inputdata[['Pmin', 'Pmax']].dropna()
    data['ormset_vol'] = inputdata[['Vmax','Vmin','Vstart']].dropna()
    data['ormset_disc'] = inputdata[['Qmin','Qmax','Q']].dropna()
    data['ormset_head'] = inputdata[['LRL','HRL','RL_start']].dropna()
    
    #Volume-head relationship:
    inputdata = pd.read_excel(data_file, sheet_name='Vol_head')
    data_volhead = {}
    data_volhead['ormset_headvol'] = inputdata[['Mm3','m3/s']].dropna()

    #Flow-head relationship:
    inputdata = pd.read_excel(data_file, sheet_name='flow')
    data_flowhead = {}
    data_flowhead['ormset_headflow'] = inputdata[['Moh','m3/s']].dropna()
    return data

topology = InputData('data/Ormset_Data.xlsx')

display(topology)

{'ormset_prod': Ormsetfoss  Pmin  Pmax
 Hydro1       0.0  43.0
 Hydro2       0.0  30.0,
 'ormset_vol': Ormsetfoss  Vmax  Vmin  Vstart
 Hydro1_vol  45.0   0.0    40.0
 Hydro2_vol  20.0   0.0    15.0,
 'ormset_disc': Ormsetfoss   Qmin  Qmax    Q
 Hydro1_disc   2.0  12.0  4.0
 Hydro2_disc   4.0  12.0  2.0,
 'ormset_head': Ormsetfoss    LRL    HRL  RL_start
 Hydro1_WL   375.0  388.0     388.0
 Hydro2_WL   326.0  331.0     331.0}

## Digital twin model to hydro power plant

In [5]:
#Definitions
start = '2018-01-01 00:00:00'
end = '2018-01-01 23:00:00'


# Define your input data and constants
L = {1: 30, 2: 20, 3: 20, 4: 30, 5: 50, 6: 80, 7: 50, 8: 90, 9: 110, 10: 150, 11: 120, 12: 80, 13: 70, 14: 80, 15: 90, 16: 160, 17: 170, 18: 150, 19: 120, 20: 100, 21: 70, 22: 60, 23: 50, 24: 40}

constants = {'eff': 0.91, 'rho': 1000, 'g': 9.81, 'H': 300}

# Create a ConcreteModel
model = pyo.ConcreteModel()

#Input data and read-in
input_data_market = read_csv_data('data/Market_price.csv')              #MARKET
market_prices_h=convert_to_dict(input_data_market, start, end, 'H') ##avg_market_price=average_value(market_prices_h)
market_prices_values = [market_prices_h[i] for i in range(1, len(market_prices_h) + 1)]     #create code to make this more efficient (in convert_to_dict for example?)

#input_data_inflow = read_csv_data('data/Data_inflow.csv')
#Q = convert_to_dict(input_data_inflow, start, end, 'H')

# Sets to model hydro topology
model.periods = pyo.Set(initialize=range(1, 25, 1))
model.plants = pyo.Set(initialize=['Hydro1', 'Hydro2'])               #uses i
model.market = pyo.Set(initialize=['DAM'])
model.discharge = pyo.Set(initialize=['Hydro1_disc', 'Hydro2_disc'])  #uses k
model.vol = pyo.Set(initialize=['Hydro1_vol', 'Hydro2_vol'])          #uses v
model.wl = pyo.Set(initialize=['Hydro1_WL','Hydro2_WL'])              #uses w

#model.market_set = pyo.RangeSet(1, len(market_prices_h))


# Hydro parameters
model.eff = pyo.Param(initialize=constants['eff'])
model.rho = pyo.Param(initialize=constants['rho'])
model.g = pyo.Param(initialize=constants['g'])
model.H = pyo.Param(initialize=constants['H'])

# Cost parameters
Fi=1000
model.Fi=pyo.Param(model.market, initialize=Fi)                  #Buy-in market price (initial market price cost)
model.Mi = pyo.Param(initialize=market_prices_values)            #Market price varying
model.Ci=pyo.Param(model.plants, initialize=parameters['prod']['Ci'])  #Initial cost for plants
model.yi=pyo.Param(model.plants, initialize=parameters['prod']['yi'])  #Variable costs for plants 

#Plant parameters
model.Pmin=pyo.Param(model.plants, initialize=topology['ormset_prod']['Pmin'])
model.Pmax=pyo.Param(model.plants, initialize=topology['ormset_prod']['Pmax'])

#Topology parameters
model.Qmin=pyo.Param(model.discharge, initialize=topology['ormset_disc']['Qmin']) 
model.Qmax=pyo.Param(model.discharge, initialize=topology['ormset_disc']['Qmax'])
model.Vmin=pyo.Param(model.vol, initialize=topology['ormset_vol']['Vmin'])
model.Vmax=pyo.Param(model.vol, initialize=topology['ormset_vol']['Vmax'])
model.v0 = pyo.Param(model.vol, initialize=topology['ormset_vol']['Vstart'])
model.LRL =pyo.Param(model.wl, initialize=topology['ormset_head']['HRL'])
model.HRL =pyo.Param(model.wl, initialize=topology['ormset_head']['LRL'])


# Variables for power produced or bought and volume, water level etc.
def p_bounds(model, i, j):
    return (model.Pmin[i], model.Pmax[i])
model.p = pyo.Var(model.plants, model.periods, bounds=p_bounds)               #Power production
model.m = pyo.Var(model.market, model.periods, within=NonNegativeReals)       #Power from market

def q_bounds(model, k, j): #flow rate
    return (model.Qmin[k], model.Qmax[k])
model.q = pyo.Var(model.discharge, model.periods, bounds=q_bounds)          #time dependent discharge which varies based on power production

def vol_bounds(model, v, j):
    return (model.Vmin[v], model.Vmax[v])
model.v = pyo.Var(model.vol, model.periods, bounds=vol_bounds)              #time dependent volume which varies based on discharge over time (convert from m3/s to MM3/day ellerno og subtraher)

#def wl_bounds(model, w, j): #water level
#    return (model.LRL[w], model.HRL[w])
#model.h = pyo.Var(model.wl, model.periods, bounds=wl_bounds)                  #time dependent water level which vares based on volume (HEAD)


#-------------------MODEL OF RESERVOIR TO BE EMPTIED------------------#
model.cumulative_q = pyo.Var(model.discharge, model.periods, within=pyo.NonNegativeReals)           #combining discharge (q [m3/s]) into m3 over given timeframe (NEED TO CHECK CONVERSIONS)
# Constraint to calculate the cumulative sum of discharges
def cumulative_q_rule(model, k, j):
    if j == 1:
        return model.cumulative_q[k, j] == model.q[k, j]
    else:
        return model.cumulative_q[k, j] == model.cumulative_q[k, j - 1] + model.q[k, j]
model.cumulative_q_cons = pyo.Constraint(model.discharge, model.periods, rule=cumulative_q_rule)
# Constraint to update volume based on cumulative sum of discharges
def volume_change_rule(model, v, j):
    if j == 1:
        return model.v[v, j] == model.v0[v] - sum(model.cumulative_q[k, j] for k in model.discharge)
    else:
        return model.v[v, j] == model.v[v, j - 1] - sum(model.cumulative_q[k, j] for k in model.discharge)
model.volume_change_cons = pyo.Constraint(model.vol, model.periods, rule=volume_change_rule)

# Constraints
def load_rule(model, j):
    return model.p['Hydro1',j] + model.p['Hydro2',j] + model.m['DAM',j] == L[j]
model.load_cons = pyo.Constraint(model.periods, rule=load_rule)

#def discharge_rule (model, j): #can't discharge forever as that will empty the reservoir. Need to include connection between discharge to volume reduction 
#    return model.q


def power_rule(model, j):  #Rule for hydro power production dependent on varying discharge #update to also vary on head
    return sum(10**-6*model.eff * model.rho * model.g * model.H * model.q[k, j] for k in model.discharge) == sum(model.p[i,j] for i in model.plants) #MW
model.power_prod = pyo.Constraint(model.periods, rule=power_rule)


# Objective function
def ObjRule(model):
    return sum(model.p[i, j] for i in model.plants for j in model.periods)

model.obj = pyo.Objective(rule=ObjRule, sense=pyo.maximize)

# Solve the model
opt = pyo.SolverFactory('gurobi', solver_io="python") 
results = opt.solve(model)

#defining dual 
model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
model.dual.display()

print("Objective Value:", pyo.value(model.obj))

# Check if the solution is optimal
if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
    power_production = {(plant, period): model.p[plant, period].value for plant in model.plants for period in model.periods}
    market_power = {(market, period): model.m[market, period].value for market in model.market for period in model.periods}
    load_values = [L[j] for j in model.periods]
    colors = ['b', 'g', 'c']  # Add 'c' for the market

    fig, ax1 = plt.subplots()
    ax1.plot(model.periods, load_values, 'r-', label='Load')
    ax1.set_xlabel('Periods [h]')
    ax1.set_ylabel('Load [MW]', color='r')
    ax1.tick_params('y', colors='r')
    ax2 = ax1.twinx()

    bottom = np.zeros(len(model.periods))  # Initialize the bottom position for stacking

    for i, plant in enumerate(model.plants):
        power_values = [power_production[plant, period] for period in model.periods]
        ax2.bar(model.periods, power_values, color=colors[i], alpha=0.5, label=f'{plant} Power Production', bottom=bottom)
        bottom += power_values  # Update the bottom position for the next set of bars

    market_power_values = [market_power['DAM', period] for period in model.periods]
    ax2.bar(model.periods, market_power_values, color='c', alpha=0.5, label='Power from market', bottom=bottom)

    ax2.set_ylabel('Power [MW]', color='b')
    ax2.tick_params('y', colors='b')
    ax1.legend(loc='upper left', bbox_to_anchor=(1.15, 1))
    ax2.legend(loc='upper left', bbox_to_anchor=(1.15, 0.85))
    plt.title('Power produced and bought vs Load')
    plt.show()
    
else:
    log_infeasible_constraints(model)
    print("Solver did not find an optimal solution.")
model.display()


# Display the model specifics
print()
model.pprint()
print()
#model.constraint.pprint()
print()
model.obj.display()
print()


model.name="unknown";
    - termination condition: infeasible
    - message from solver: <undefined>
dual : Direction=Suffix.IMPORT, Datatype=Suffix.FLOAT
    Key : Value
ERROR: evaluating object as numeric value: p[Hydro1,1]
        (object: <class 'pyomo.core.base.var._GeneralVarData'>)
    No value for uninitialized NumericValue object p[Hydro1,1]
ERROR: evaluating object as numeric value: obj
        (object: <class 'pyomo.core.base.objective.ScalarObjective'>)
    No value for uninitialized NumericValue object p[Hydro1,1]


ValueError: No value for uninitialized NumericValue object p[Hydro1,1]

### Useful code for later 

In [ ]:
#OBTAIN DICTIONARY VALUES THAT CAN BE USED IN PARAMETER MODELLIGN

a = market_prices_h
values = []
for value in a.values():
    values.append(value)
print(values)

[26.33, 26.43, 26.1, 24.7, 24.74, 24.67, 24.98, 25.61, 25.48, 25.54, 26.07, 26.12, 26.16, 26.19, 26.19, 26.46, 27.03, 27.43, 27.29, 26.96, 26.7, 26.47, 26.09, 25.77]
